In [0]:
# %run ../00_config

In [0]:
print("過去視聴ログのCSVファイルを作成します...")

In [0]:
df_contents = spark.read.csv(
    f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/ife_contents/ife_contents.csv",
    header=True
)
print(df_contents.count())

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import (
    col, to_timestamp, to_date,
    avg, round as spark_round, expr
)
import random, datetime

# ============================================================
# テーブル: ife_play_logs（機内エンタメ視聴ログ）サンプル
# ============================================================

# ---------- 設定 ----------
N_CONTENTS        = 25      # コンテンツ総数　←重要：コンテンツマスタの数と合わせる
MIN_VIEWS_PER_BK  = 2       # ★ 1 予約あたり最小視聴セッション数
MAX_VIEWS_PER_BK  = 8       # ★ 〃             最大視聴セッション数
MIN_SEC           = 30      # 1 セッション最短視聴 30 秒
MAX_SEC           = 2*60*60 # 〃 最長視聴 2 時間 (7,200 秒)

# ---------- コンテンツマスタを読み込む ----------
df_contents = spark.read.csv(
    f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/ife_contents/ife_contents.csv",
    header=True
)
print(df_contents)

# content_id (カテゴリ, duration_sec) を辞書化
content_meta = {
    int(row.content_id): (row.content_category, int(row.duration_sec))
    for row in df_contents.collect()
}

# ---------- フライト予約履歴を読み込む  # ★ add ----------
df_booking = spark.read.csv(
    f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/flight_booking/flight_booking.csv",
    header=True
).withColumn("flight_date", to_date(col("flight_date")))

# ---------- 日時生成パラメータ ----------
start_dt  = datetime.datetime(2024, 1, 1, 0, 0, 0)
end_dt    = datetime.datetime(2025, 12, 31, 23, 59, 59)
total_sec = int((end_dt - start_dt).total_seconds())

def rand_play_start(bk_date: datetime.date) -> datetime.datetime:
    """
    視聴開始日時をランダム生成（フライト当日 0:00–23:59 の間で発生）
    """
    return datetime.datetime.combine(
        bk_date,
        datetime.time()
    ) + datetime.timedelta(seconds=random.randint(0, 86399))

def rand_watch_ratio() -> float:
    """視聴割合 β(α=3,β=3) で多様性を確保（平均 ≒ 0.5）"""
    return random.betavariate(8, 1) if random.random() < .25 else random.betavariate(3, 3)

# ---------- 行レコードを組み立て ----------
rows = []
for bk in df_booking.collect():
    uid = int(bk.user_id)

    # 予約 1 件あたり 2〜8 セッションを発生させる ★
    for _ in range(random.randint(MIN_VIEWS_PER_BK, MAX_VIEWS_PER_BK)):
        # ---- 視聴コンテンツを決定 ----
        cid = random.randint(1, N_CONTENTS)
        cat, duration_sec = content_meta[cid]

        # ---- 視聴秒数を決定 ----
        watch_ratio = rand_watch_ratio()
        play_sec    = max(MIN_SEC, int(duration_sec * watch_ratio))
        play_sec    = min(play_sec, MAX_SEC)

        # ---- 視聴開始・終了時刻 ----
        play_start = rand_play_start(bk.flight_date)
        play_end   = play_start + datetime.timedelta(seconds=play_sec)

        rows.append(
            Row(
                user_id          = uid,                # 会員ID（数値型）
                flight_id        = bk.flight_id,       # フライトID
                route_id         = bk.route_id,        # ルート
                flight_date      = bk.flight_date,     # フライト日
                content_id       = cid,                # コンテンツID
                content_category = cat,                # カテゴリ
                duration_sec     = duration_sec,       # 作品尺（秒）
                play_sec         = play_sec,           # 視聴秒数
                play_start_at    = play_start.strftime("%Y-%m-%d %H:%M:%S"),
                play_end_at      = play_end.strftime("%Y-%m-%d %H:%M:%S"),
                play_start_date  = play_start.date().isoformat()
            )
        )

# ---------- DataFrame 化 ----------
df = spark.createDataFrame(rows) \
    .withColumn("play_start_at", to_timestamp(col("play_start_at"))) \
    .withColumn("play_end_at",   to_timestamp(col("play_end_at"))) \
    .withColumn("play_start_date", to_date(col("play_start_date"))) \
    .withColumn("flight_date",     to_date(col("flight_date")))      # ★ add

# ---------- 検証用: pct_watched を派生（保存時はドロップ） ----------
df_debug = df.withColumn(
    "pct_watched",
    spark_round(expr("play_sec / duration_sec * 100"), 2)
)

# ---------- 動作確認 ----------
print("視聴レコード数:", df.count())      # 例: ≒ 60,000〜160,000 ほど増えるはず
df.printSchema()
# df_debug.show(10, truncate=False)

# ---------- Delta テーブル保存 ----------
# df.write.format("delta")\
#   .option("comment", "機内エンターテイメントの過去視聴ログ（フライト紐付け版）")\
#   .mode("overwrite")\
#   .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_ife_play_logs")

# ---------- CSV 出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/ife_play_logs/ife_play_logs.csv"
df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("過去視聴ログのCSVファイルを作成しました！")